# KONTEKSTNO NEODVISNE GRAMATIKE ZA ODKRIVANJE ENAČB

Odkrivanje enačb s pomočjo gramatik in njihova analiza.

## 1. GENERIRANJE PRIMEROV

In [1]:
import pandas as pd
import numpy as np
from ProGED.parameter_estimation import fit_models
from ProGED.generate import generate_models
from ProGED.generators.grammar import GeneratorGrammar
from math import sqrt, cos, sin, tan, exp

Za vsako izmed enačb zgeneriramo 100 primerov vhodnih in izhodne spremenljivke.

In [2]:
PODATKOVJA = ["proznostna.csv", "enakomerno_pospeseno.csv", "kosinusni.csv", "nihanje.csv"]


def generiraj_proznostna(primeri):
    """
    Generira podatke za enačbo W = 1/2 * k * x^2
    (Ciljna spremenljivka je W.)
    :param primeri:
    :return:
    """
    np.random.seed(1234)
    podatki = {"W": [], "k": [], "x": []}
    for i in range(primeri):
        k, x = np.random.rand(2)
        W = 1/2 * k * x**2
        podatki["W"].append(W)
        podatki["k"].append(k)
        podatki["x"].append(x)
    df = pd.DataFrame(podatki)
    df.to_csv("proznostna.csv", sep=",", index=None)
    
    
def generiraj_enakomerno_pospeseno(primeri):
    """
    Generira podatke za enačbo s = 1/2at^2 + v0t
    (Ciljna spremenljivka je s.)
    :param primeri:
    :return:
    """
    np.random.seed(1234)
    podatki = {"s": [], "a": [], "t": [], "v0": []}
    for i in range(primeri):
        a, t, v0 = np.random.rand(3)
        s = 1/2 * a * t**2 + v0 * t
        podatki["s"].append(s)
        podatki["a"].append(a)
        podatki["t"].append(t)
        podatki["v0"].append(v0)
    df = pd.DataFrame(podatki)
    df.to_csv("enakomerno_pospeseno.csv", sep=",", index=None)
    
    
def generiraj_kosinusni(primeri):
    """
    Generira podatke za enačbo c = sqrt(a^2 + b^2 - 2abcos(gamma))
    (Ciljna spremenljivka je c.)
    :param primeri:
    :return:
    """
    np.random.seed(1234)
    podatki = {"c": [], "a": [], "b": [], "g": []}
    for i in range(primeri):
        a, b, g = np.random.rand(3)
        c = sqrt(a**2 + b**2 - 2*a*b*cos(g))
        podatki["c"].append(c)
        podatki["a"].append(a)
        podatki["b"].append(b)
        podatki["g"].append(g)
    df = pd.DataFrame(podatki)
    df.to_csv("kosinusni.csv", sep=",", index=None)
    
    
def generiraj_nihanje(primeri):
    """
    Generira podatke za enačbo x = x0 sin(omega t + alpha0).
    (Ciljna spremenljivka je x.)
    :param primeri:
    :return:
    """
    np.random.seed(1234)
    podatki = {"x": [], "x0": [], "omega": [], "t": [], "alpha0": []}
    for i in range(primeri):
        x0, omega, t, alpha0 = np.random.rand(4)
        x = x0 * sin(omega * t + alpha0)
        podatki["x"].append(x)
        podatki["x0"].append(x0)
        podatki["omega"].append(omega)
        podatki["t"].append(t)
        podatki["alpha0"].append(alpha0)
    df = pd.DataFrame(podatki)
    df.to_csv("nihanje.csv", sep=",", index=None)

In [3]:
n_primerov = 100
generiraj_proznostna(n_primerov)
generiraj_enakomerno_pospeseno(n_primerov)
generiraj_kosinusni(n_primerov)
generiraj_nihanje(n_primerov)

In [4]:
def nalozi_podatke(ime, ciljna_spremenljivka):
    df = pd.read_csv(ime)
    stolpci = list(df.columns)
    indeks_ciljne = stolpci.index(ciljna_spremenljivka)
    return np.array(df), indeks_ciljne


def resi_z_gramatiko(podatki_datoteka, ime_ciljne, gramatika, simboli, n_modelov):
    podatki, indeks_ciljne = nalozi_podatke(podatki_datoteka, ime_ciljne)
    grammar = GeneratorGrammar(gramatika)
    print("##################### Kandidati: #####################")
    modeli = generate_models(grammar, simboli, strategy_settings={"N": n_modelov}, verbosity=1)
    modeli = fit_models(modeli, podatki, indeks_ciljne)

    print("##################### Rezultati: #####################")
    trojke = [(m.get_error(), m.p, str(m.get_full_expr())) for m in modeli]
    trojke.sort()
    for e, p, m in trojke:
        print(f"napaka modela: {e:.4e}, verjetnost modela: {p:.4e}, model: {m}")


### GENERIRANJE GRAMATIK
Funkcije za generiranje gramtik. Večina kode je pobrane iz repozitorija ProGED na GitHub-u. Bomo pa končno funkcijo prilagodili za naše potrebe.
Dodatno bomo potem ročno odstranili nekatera prepisovalne pravila v posameznih primerih, kjer so neuporabna.

In [5]:
def grammar_from_template (template_name, generator_settings):
    if template_name in GRAMMAR_LIBRARY:
        grammar_str = GRAMMAR_LIBRARY[template_name](**generator_settings)
        return GeneratorGrammar(grammar_str)

def construct_right (right = "a", prob = 1):
    return right + " [" + str(prob) + "]"

def construct_production (left = "S", items = ["a"], probs=[1]):
    if not items:
        return ""
    else:
        return "\n" + left + " -> " + construct_right_distribution (items=items, probs=probs)

def construct_right_distribution (items=[], probs=[]):
    p = np.array(probs)/np.sum(probs)
    S = construct_right(right=items[0], prob=p[0])
    for i in range(1, len(items)):
        S += " | " + construct_right(right=items[i], prob=p[i])
    return S


#########################################################################################################
#########################################################################################################
#########################################################################################################
def unit_to_string (unit, unit_symbols=["m", "s", "kg"]):
    return "".join([unit_symbols[i]+str(unit[i]) for i in range(len(unit))])

def string_to_unit (unit_string, unit_symbols=["m", "s", "kg"]):
    u = []
    for i in range(len(unit_symbols)-1):
        split = unit_string.split(unit_symbols[i])[1].split(unit_symbols[i+1])
        u += [int(split[0])]
    u += [int(split[1])]
    return u

def units_dict (variables, units, dimensionless = [0,0,0], target_variable_unit = [0,0,0]):
    dictunits = {}
    for i in range(len(variables)):
        unit_string = unit_to_string(units[i])
        if unit_string in dictunits:
            dictunits[unit_string] += [variables[i]]
        else:
            dictunits[unit_string] = [variables[i]]
    if unit_to_string(dimensionless) not in dictunits:
        dictunits[unit_to_string(dimensionless)] = []
    #if unit_to_string(unit_to_string(units[target_variable_unit_index])) not in dictunits:
    #    dictunits[unit_to_string(units[target_variable_unit_index])] = []
    if unit_to_string(target_variable_unit) not in dictunits:
        dictunits[unit_to_string(target_variable_unit)] = []
    return dictunits

def unit_conversions(units_dict, order=1):
    conversions = {}
    #units = np.array([np.fromstring(unit.strip("[").strip("]").strip(), sep=",", dtype=int) for unit in list(units_dict.keys())])
    units = np.array([string_to_unit(unit) for unit in list(units_dict.keys())])
    for i in range(len(units)):
        conversions_mul = []
        conversions_div = []
        for j in range(len(units)):
            for k in range(len(units)):
                if np.array_equal(units[i], units[j] + units[k]):
                    if [j,k] not in conversions_mul and [k,j] not in conversions_mul:
                        conversions_mul += [[j,k]]
                if np.array_equal(units[i], units[j] - units[k]):
                    if [j,k] not in conversions_div:
                        conversions_div += [[j,k]]
                if np.array_equal(units[i], units[k]- units[j]):
                    if [k,j] not in conversions_div:
                        conversions_div += [[k,j]]
        conversions[str(i)+"*"] = conversions_mul
        conversions[str(i)+"/"] = conversions_div
    return conversions, units

def probs_uniform(items, A=1):
    if len(items) > 0:
        return [A/len(items)]*len(items)
    else:
        return []

In [6]:
def generiraj_gramatiko(variables=["'A'", "'b'", "'c'"], 
                        p_sum = [0.2, 0.2, 0.6], 
                        p_mul = [0.2, 0.2, 0.6], 
                        p_rec=[0.2, 0.4, 0.4], 
                        functions=["sin", "cos", "sqrt"], p_functs=[0.7, 0.1, 0.1, 0.1], 
                        units = [[2,-2,1], [1,0,0], [-1,0,0], [2,-2,1]], 
                        target_variable_unit_index = -1, 
                        dimensionless = [0,0,0]):
    
    target_variable_unit = units[target_variable_unit_index]
    dictunits = units_dict(variables, units, dimensionless = dimensionless, target_variable_unit = target_variable_unit)
    conversions, unique_units = unit_conversions(dictunits)
    strunits = [unit_to_string(unit) for unit in unique_units]
    
    # konstruiramo gramatiko
    grammar = construct_production(left="S", items=["E_" + unit_to_string(target_variable_unit)], probs=[1.0])
    
    # v gramatiko po vrsti dodajamo pravila s "E_ ", "F_ " in "T_ "
    for i in range(len(unique_units)):          
        right_sum = ["E_" + strunits[i] + "'+'" + "F_" + strunits[i]]
        right_sub = ["E_" + strunits[i] + "'-'" + "F_" + strunits[i]]
        right_Fid = ["F_" + strunits[i]]
        grammar += construct_production(left="E_" + strunits[i], 
                                        items = right_sum + right_sub + right_Fid,
                                        probs = p_sum)
        
        right_mul = ["F_" + strunits[conv[0]] + "'*'" + "T_" + strunits[conv[1]] for conv in conversions[str(i)+"*"]]
        right_div = ["F_" + strunits[conv[0]] + "'/'" + "T_" + strunits[conv[1]] for conv in conversions[str(i)+"/"]]
        right_Tid = ["T_" + strunits[i]]
        probs_mul = probs_uniform(right_mul, A=p_mul[0])
        probs_div = probs_uniform(right_div, A=p_mul[1])
        grammar += construct_production(left="F_" + strunits[i], 
                                        items = right_mul + right_div + right_Tid,
                                        probs = probs_mul + probs_div + [p_mul[2]])
        
        if strunits[i] == unit_to_string(dimensionless):
            right_recur = ["F"]
        else:
            right_recur = ["'('" + "E_" + strunits[i] + "')'"]
        right_var = dictunits[unit_to_string(unique_units[i])]
        right_const = ["'C'"]
        probs_vars = probs_uniform(dictunits[strunits[i]], A=p_rec[1])
        if strunits[i] != unit_to_string(dimensionless):
            grammar += construct_production(left="T_" + strunits[i], 
                                            items = right_recur + right_var,
                                            probs = [p_rec[0]] + probs_vars)
        
        # dodamo konstanto
        if strunits[i] == unit_to_string(dimensionless):
            grammar += construct_production(left="T_" + strunits[i], 
                                            items = right_recur + right_var + right_const,
                                            probs = [p_rec[0]] + probs_vars + [p_rec[2]])
        
        # dodamo še funkcije
        if strunits[i] == unit_to_string(dimensionless):
            right_F = ["'('" + "E_" + strunits[i] + "')'"] + ["'"+f+"('" + "E_"+strunits[i] + "')'" for f in functions]
            grammar += construct_production(left = "F", items=right_F, probs=p_functs)

    return grammar

GRAMMAR_LIBRARY = {"universal-dim": generiraj_gramatiko}

Poglejmo si primer splošne gramatike, ki jo zgenerira naš program.

In [7]:
gramatika_splosna = grammar_from_template("universal-dim", {"variables":["'W'", "'k'", "'x'"]})
print(gramatika_splosna)

Grammar with 43 productions (start state = S)
    S -> E_m2s-2kg1 [1.0]
    E_m2s-2kg1 -> E_m2s-2kg1 '+' F_m2s-2kg1 [0.2]
    E_m2s-2kg1 -> E_m2s-2kg1 '-' F_m2s-2kg1 [0.2]
    E_m2s-2kg1 -> F_m2s-2kg1 [0.6]
    F_m2s-2kg1 -> F_m2s-2kg1 '*' T_m0s0kg0 [0.2]
    F_m2s-2kg1 -> F_m2s-2kg1 '/' T_m0s0kg0 [0.2]
    F_m2s-2kg1 -> T_m2s-2kg1 [0.6]
    T_m2s-2kg1 -> '(' E_m2s-2kg1 ')' [0.333333]
    T_m2s-2kg1 -> 'W' [0.666667]
    E_m1s0kg0 -> E_m1s0kg0 '+' F_m1s0kg0 [0.2]
    E_m1s0kg0 -> E_m1s0kg0 '-' F_m1s0kg0 [0.2]
    E_m1s0kg0 -> F_m1s0kg0 [0.6]
    F_m1s0kg0 -> F_m1s0kg0 '*' T_m0s0kg0 [0.2]
    F_m1s0kg0 -> F_m1s0kg0 '/' T_m0s0kg0 [0.1]
    F_m1s0kg0 -> F_m0s0kg0 '/' T_m-1s0kg0 [0.1]
    F_m1s0kg0 -> T_m1s0kg0 [0.6]
    T_m1s0kg0 -> '(' E_m1s0kg0 ')' [0.333333]
    T_m1s0kg0 -> 'k' [0.666667]
    E_m-1s0kg0 -> E_m-1s0kg0 '+' F_m-1s0kg0 [0.2]
    E_m-1s0kg0 -> E_m-1s0kg0 '-' F_m-1s0kg0 [0.2]
    E_m-1s0kg0 -> F_m-1s0kg0 [0.6]
    F_m-1s0kg0 -> F_m-1s0kg0 '*' T_m0s0kg0 [0.2]
    F_m-1s0kg0 

## 2.1 ENERGIJA PROŽNOSTNEGA TELESA

Generiramo splošno gramatiko ter jo potem popravimo glede na opisana pravila v poročilu.

In [8]:
# Pozor! Konstante so lahko le brezdimenzijske.

grammar_proznostna = grammar_from_template("universal-dim", {"variables": ["'W'", "'k'", "'x'"], 
                                                  "units": [[2,-2,1], [0,-2,1], [1,0,0], [2,-2,1]], 
                                                  "target_variable_unit_index": -1})
# print(grammar_proznostna)

In [9]:
gramatika_proznostna = """S -> E_m2s-2kg1 [1.0]
    E_m2s-2kg1 -> E_m2s-2kg1 '+' F_m2s-2kg1 [0.2]
    E_m2s-2kg1 -> E_m2s-2kg1 '-' F_m2s-2kg1 [0.2]
    E_m2s-2kg1 -> F_m2s-2kg1 [0.6]
    F_m2s-2kg1 -> F_m2s-2kg1 '*' T_m0s0kg0 [0.2]
    F_m2s-2kg1 -> F_m0s-2kg1 '*' T_m1s0kg0 '*' T_m1s0kg0 [0.4]
    F_m2s-2kg1 -> T_m2s-2kg1 [0.4]
    T_m2s-2kg1 -> '(' E_m2s-2kg1 ')' [1.0]
    
    E_m0s-2kg1 -> E_m0s-2kg1 '+' F_m0s-2kg1 [0.2]
    E_m0s-2kg1 -> E_m0s-2kg1 '-' F_m0s-2kg1 [0.2]
    E_m0s-2kg1 -> F_m0s-2kg1 [0.6]
    F_m0s-2kg1 -> F_m0s-2kg1 '*' T_m0s0kg0 [0.4]
    F_m0s-2kg1 -> T_m0s-2kg1 [0.6]
    T_m0s-2kg1 -> '(' E_m0s-2kg1 ')' [0.2]
    T_m0s-2kg1 -> 'k' [0.8]

    E_m1s0kg0 -> E_m1s0kg0 '+' F_m1s0kg0 [0.2]
    E_m1s0kg0 -> E_m1s0kg0 '-' F_m1s0kg0 [0.2]
    E_m1s0kg0 -> F_m1s0kg0 [0.6]
    F_m1s0kg0 -> F_m1s0kg0 '*' T_m0s0kg0 [0.4]
    F_m1s0kg0 -> T_m1s0kg0 [0.6]
    T_m1s0kg0 -> '(' E_m1s0kg0 ')' [0.2]
    T_m1s0kg0 -> 'x' [0.8]
    
    E_m0s0kg0 -> E_m0s0kg0 '+' F_m0s0kg0 [0.2]
    E_m0s0kg0 -> E_m0s0kg0 '-' F_m0s0kg0 [0.2]
    E_m0s0kg0 -> F_m0s0kg0 [0.6]
    F_m0s0kg0 -> F_m0s0kg0 '*' T_m0s0kg0 [0.4]
    F_m0s0kg0 -> T_m0s0kg0 [0.6]
    T_m0s0kg0 -> F [0.333333]
    T_m0s0kg0 -> 'C' [0.666667]
    
    F -> '(' E_m0s0kg0 ')' [0.7]
    F -> 'sin(' E_m0s0kg0 ')' [0.1]
    F -> 'cos(' E_m0s0kg0 ')' [0.1]
    F -> 'sqrt(' E_m0s0kg0 ')' [0.1]"""

simboli_proznostna = {"x": ['k', 'x'], "start": "S", "const": "C"}

In [10]:
resi_z_gramatiko("proznostna.csv", "W", gramatika_proznostna, simboli_proznostna, 20)            

##################### Kandidati: #####################
C0*k*x**2
2*C0**2*k*x**2 + C1*k*x**2 + k*x**2
3*C0*k*x**2 - k*x**2
k*x**2
5*C0*k*x**2 + C1*x**2*(C2*(2*C3*k + k) + 2*k) + k*x**2
C0*(2*C1*k*x**2 + k*x*(C2*x + x)) + x**2*(C3*k + k)
C0*(2*C1*k*x**2 + k*x*(C2*x + x)) + x**2*(C3*k + k)
C0*k*x**2 + C1*x**2*(C2*k + 2*k)
C0*k*x**2 + C1*x**2*(C2*k + 2*k)
C0*x*(C1*k - 3*k)*(C2*x - x)
4*C0*k*x**2 + C1*k*x*(C2*x + x) + C16*(C17*k*x**2 + C18*k*x*(C19*x - x) + C20*x**2*(C21*k + k) - k*x*(C22*x + x)) + C3*k*x*(2*C4*x + x) + C5*(C6*k*x**2 + C7*(C11*x**2*(4*C12*k - k) + C13*(C14*x**2*(C15*k + 2*k) + k*x**2) + 2*C8*k*x**2 + C9*k*x*(C10*x - x) - 4*k*x**2)) + k*x**2
3*C0*k*x**2 + C1*(C2*k*x**2 + k*x**2) - 3*k*x**2
2*C0*k*x**2 + C1*k*x*(2*C2*x - x) + C3*(C4*k*x**2 + C5*x**2*(C6*k + C7*(C8*k - 2*k))) + C9*(2*C10*k*x**2 + k*x**2) + x**2*(C11*k + k)
C0*k*x**2 + x**2*(C1*k - k)
2*C0*k*x**2 + k*x**2
2*C0*k*x**2 + k*x**2
C0*k*x*(C1*x + x)
2*C0**2*k*x**2
2*C0*k*x**2
k*x**2 + x**2*(C0*k + 2*k)
##############

## 2.2 PREPOTOVANA POT PRI ENAKOMERNEM POSPEŠENEM GIBANJU

In [11]:
grammar_enakomerno = grammar_from_template("universal-dim", {"variables": ["'s'", "'a'", "'t'", "'v0'"], 
                                                  "units": [[1,0,0], [1,-2,0], [0,1,0], [1,-1,0], [1,0,0]], 
                                                  "target_variable_unit_index": -1})
# print(grammar_enakomerno)

In [12]:
gramatika_enakomerno = """ S -> E_m1s0kg0 [1.0] 
    E_m1s0kg0 -> E_m1s0kg0 '+' F_m1s0kg0 [0.3]
    E_m1s0kg0 -> E_m1s0kg0 '-' F_m1s0kg0 [0.3]
    E_m1s0kg0 -> F_m1s0kg0 [0.4]
    F_m1s0kg0 -> F_m1s0kg0 '*' T_m0s0kg0 [0.2]
    F_m1s0kg0 -> F_m0s1kg0 '*' T_m1s-1kg0 [0.2]
    F_m1s0kg0 -> F_m1s-2kg0 '*' T_m0s1kg0 '*' T_m0s1kg0 [0.4]
    F_m1s0kg0 -> T_m1s0kg0 [0.2]
    T_m1s0kg0 -> '(' E_m1s0kg0 ')' [1.0]
    
    E_m1s-2kg0 -> E_m1s-2kg0 '+' F_m1s-2kg0 [0.3]
    E_m1s-2kg0 -> E_m1s-2kg0 '-' F_m1s-2kg0 [0.3]
    E_m1s-2kg0 -> F_m1s-2kg0 [0.4]
    F_m1s-2kg0 -> F_m1s-2kg0 '*' T_m0s0kg0 [0.3]
    F_m1s-2kg0 -> F_m1s-1kg0 '/' T_m0s1kg0 [0.1]
    F_m1s-2kg0 -> T_m1s-2kg0 [0.6]
    T_m1s-2kg0 -> '(' E_m1s-2kg0 ')' [0.1]
    T_m1s-2kg0 -> 'a' [0.9]

    E_m0s1kg0 -> E_m0s1kg0 '+' F_m0s1kg0 [0.1]
    E_m0s1kg0 -> E_m0s1kg0 '-' F_m0s1kg0 [0.1]
    E_m0s1kg0 -> F_m0s1kg0 [0.8]
    F_m0s1kg0 -> F_m0s1kg0 '*' T_m0s0kg0 [0.2]
    F_m0s1kg0 -> F_m1s0kg0 '/' T_m1s-1kg0 [0.1]
    F_m0s1kg0 -> F_m1s-1kg0 '/' T_m1s-2kg0 [0.1]
    F_m0s1kg0 -> T_m0s1kg0 [0.6]
    T_m0s1kg0 -> '(' E_m0s1kg0 ')' [0.1]
    T_m0s1kg0 -> 't' [0.9]
    
    E_m1s-1kg0 -> E_m1s-1kg0 '+' F_m1s-1kg0 [0.1]
    E_m1s-1kg0 -> E_m1s-1kg0 '-' F_m1s-1kg0 [0.1]
    E_m1s-1kg0 -> F_m1s-1kg0 [0.8]
    F_m1s-1kg0 -> F_m1s-2kg0 '*' T_m0s1kg0 [0.3]
    F_m1s-1kg0 -> F_m1s-1kg0 '*' T_m0s0kg0 [0.2]
    F_m1s-1kg0 -> F_m1s0kg0 '/' T_m0s1kg0 [0.1]
    F_m1s-1kg0 -> T_m1s-1kg0 [0.4]
    T_m1s-1kg0 -> '(' E_m1s-1kg0 ')' [0.1]
    T_m1s-1kg0 -> 'v0' [0.9]

    E_m0s0kg0 -> F_m0s0kg0 [1.0]
    F_m0s0kg0 -> F_m0s0kg0 '*' T_m0s0kg0 [0.2]
    F_m0s0kg0 -> T_m0s0kg0 [0.8]
    T_m0s0kg0 -> F [0.3]
    T_m0s0kg0 -> 'C' [0.7]
    F -> '(' E_m0s0kg0 ')' [0.7]
    F -> 'sin(' E_m0s0kg0 ')' [0.1]
    F -> 'cos(' E_m0s0kg0 ')' [0.1]
    F -> 'sqrt(' E_m0s0kg0 ')' [0.1]"""

simboli_enakomerno = {"x": ['a', 't', 'v0'], "start": "S", "const": "C"}

In [13]:
resi_z_gramatiko("enakomerno_pospeseno.csv", "s", gramatika_enakomerno, simboli_enakomerno, 100)            

##################### Kandidati: #####################
2*C0*a*t**2 + C1*(a*t**2 - a*t*(C2*v0/a + 2*t)) + C3*v0**2/a + t*v0
a*t**2
C0*t*v0 - 2*a*t**2 + t*v0
C0*a*t**2 + a*t**2
C0*a*t**2 + a*t**2
C0*t**2*v0/(C1*a*t**2/v0 + t) + C2*t*v0 - 4*a*t**2 + t*v0
2*C0*t*v0 + t*v0
C0*t*v0
C0*t*v0
C0*a*t**2 + C1*t*v0 - a*t**2 + v0**2/a
2*C0*a*t**2 + C1*t*v0 + C2*(2*C3*t*v0 + t*v0) - 3*a*t**2 - t*v0
C0*a*t**2
3*C0*a*t**2 + C1*(C2*a*t**2 + C3*t*v0 - t*v0) + a*t**2
3*C0*a*t**2 + C1*t**2*(C2*a + C3*v0/(C4*t + t) + C5*v0/t + 3*a) + 2*C6*t*v0 + C7*t*(C8*v0/t - a)*(-a*t + 2*v0 + (C10*a*t*(2*C11*t + t) + C12*v0**2/(C13*a + C14*v0/t - a) + C15*(C16*a*t**2*v0/(C17*a*t + C18*v0) + C19*a*t**2) + C9*a*t**2 - a*t**2)/t)/a + t*v0
2*C0*a*t**2
2*C0*a*t**2 + C1*t*v0
C0*a*t**2 + 2*C1*t*v0
C0*t*v0 + t*v0
2*C0*a*t**2 + C1*t*v0 + t*v0
C0*(C1*a*t**2 + 2*a*t**2 - a*t*(a*t**2/v0 + t)) + C2*(3*C3*a*t**2 + C4*t**2*(C5*a + C6*v0/t) + C7*t*v0 + a*t**2 + t**2*(C8*a + C9*v0/t))
C0*(C1*a*t**2 + 2*a*t**2 - a*t*(a*t**2/v0 + t)) + C2

## 2.3 KOSINUSNI IZREK

In [14]:
grammar_kosinusni = grammar_from_template("universal-dim", {"variables": ["'c'", "'a'", "'b'", "'g'"],
                                                            "units": [[1,0,0], [1,0,0], [1,0,0], [0,0,0], [1,0,0]], 
                                                            "target_variable_unit_index": -1})
# print(grammar_kosinusni)

In [15]:
gramatika_kosinusni = """S -> E_m1s0kg0 [0.2] | 'sqrt (' E_m2s0kg0 ')' [0.8] 
    E_m2s0kg0 -> E_m2s0kg0 '+' F_m2s0kg0 [0.1]
    E_m2s0kg0 -> E_m2s0kg0 '-' F_m2s0kg0 [0.1]
    E_m2s0kg0 -> F_m2s0kg0 [0.8]
    F_m2s0kg0 -> F_m2s0kg0 '*' T_m0s0kg0 [0.1]
    F_m2s0kg0 -> F_m1s0kg0 '*' T_m1s0kg0 [0.5]
    F_m2s0kg0 -> F_m2s0kg0 '/' T_m0s0kg0 [0.1]
    F_m2s0kg0 -> T_m2s0kg0 [0.3]
    T_m2s0kg0 -> '(' E_m2s0kg0 ')' [1.0]

    E_m1s0kg0 -> E_m1s0kg0 '+' F_m1s0kg0 [0.1]
    E_m1s0kg0 -> E_m1s0kg0 '-' F_m1s0kg0 [0.1]
    E_m1s0kg0 -> F_m1s0kg0 [0.8]
    F_m1s0kg0 -> F_m1s0kg0 '*' T_m0s0kg0 [0.5]
    F_m1s0kg0 -> F_m1s0kg0 '/' T_m0s0kg0 [0.1]
    F_m1s0kg0 -> T_m1s0kg0 [0.4]
    T_m1s0kg0 -> '(' E_m1s0kg0 ')' [0.1]
    T_m1s0kg0 -> 'a' [0.45]
    T_m1s0kg0 -> 'b' [0.45]
    
    E_m0s0kg0 -> E_m0s0kg0 '+' F_m0s0kg0 [0.1]
    E_m0s0kg0 -> E_m0s0kg0 '-' F_m0s0kg0 [0.1]
    E_m0s0kg0 -> F_m0s0kg0 [0.8]
    F_m0s0kg0 -> F_m0s0kg0 '*' T_m0s0kg0 [0.1]
    F_m0s0kg0 -> F_m0s0kg0 '/' T_m0s0kg0 [0.1]
    F_m0s0kg0 -> T_m0s0kg0 [0.8]
    T_m0s0kg0 -> F [0.3]
    T_m0s0kg0 -> 'g' [0.4]
    T_m0s0kg0 -> 'C' [0.3]
    
    F -> '(' E_m0s0kg0 ')' [0.2]
    F -> 'cos(' E_m0s0kg0 ')' [0.8]"""

simboli_kosinusni = {"x": ['a', 'b', 'g'], "start": "S", "const": "C"}

In [16]:
resi_z_gramatiko("kosinusni.csv", "c", gramatika_kosinusni, simboli_kosinusni, 100)            

##################### Kandidati: #####################
sqrt(C0*b**2*g**2)
sqrt(a**2)
sqrt(C0*a**2/g)
sqrt(b*(C0*a*g/cos(g) + C1*b*g + a)*cos(g))
sqrt(C0*a*b)
sqrt(C0*a*b - a*b)
sqrt(C0*a*b*g*cos(g)**2*cos(cos(g + cos(C1 + cos(g)))) + C2*(C3*a**2 + b**2)*cos(g) + a**2*g + b**2*g**2 + b**2*g*cos(g))
sqrt(C0*a**2*g**3/cos(g))
sqrt(C0*a*b*cos(g))
sqrt(C0*a**2*cos(C1*g) + b**2/g)
sqrt(a*b/g)
sqrt(a**2 + a*b*g + b**2)
sqrt(C0*a**2 + a**2*g + a*b - b**2*g**2)
sqrt(a*b)
sqrt(a**2*g)
sqrt(C0*a**2 + a**2 - a*b*g + b*(C1*a*g + C2*b - a)/g)
b*g
-a*cos(g)*cos(cos(C0*g)) + b
b
sqrt(C0*a*b*g*cos(g))
sqrt(C0*a*b*cos(g)/g**2 + C1*a*g*(a*g + b)*cos(C2 + g)*cos(cos(C3 + cos(C4 - cos(g))))/cos(cos(C5*cos(g))))
sqrt(C0*a*b*cos(g)/g**2 + C1*a*g*(a*g + b)*cos(C2 + g)*cos(cos(C3 + cos(C4 - cos(g))))/cos(cos(C5*cos(g))))
C0*a
sqrt(C0*a*b + C1*b**2)
sqrt(a*b*cos(cos(g)))
sqrt(a*b*cos(cos(g)))
sqrt(b**2)
sqrt(C0*a*b/cos(C1*cos(cos(C2*g))))
sqrt(C0*a*b/cos(C1*cos(cos(C2*g))))
C0*b
sqrt(b*(C0*b*g**2 - a*cos(cos(g)

## 2.4 SINUSNO NIHANJE

In [17]:
grammar_nihanje = grammar_from_template("universal-dim", {"variables": ["'x'", "'x0'", "'omega'", "'t'", "'alpha0'"],
                                                          "units": [[1,0,0], [1,0,0], [0,-1,0], 
                                                                    [0,1,0], [0,0,0], [1,0,0]],
                                                          "target_variable_unit_index": -1})
# print(grammar_nihanje)

In [18]:
gramatika_nihanje = """S -> E_m1s0kg0 [1.0]
    E_m1s0kg0 -> E_m1s0kg0 '+' F_m1s0kg0 [0.1]
    E_m1s0kg0 -> E_m1s0kg0 '-' F_m1s0kg0 [0.1]
    E_m1s0kg0 -> F_m1s0kg0 [0.8]
    F_m1s0kg0 -> F_m1s0kg0 '*' T_m0s0kg0 [0.4]
    F_m1s0kg0 -> T_m1s0kg0 [0.6]
    T_m1s0kg0 -> '(' E_m1s0kg0 ')' [0.2]
    T_m1s0kg0 -> 'x0' [0.8]
    
    E_m0s-1kg0 -> E_m0s-1kg0 '+' F_m0s-1kg0 [0.1]
    E_m0s-1kg0 -> E_m0s-1kg0 '-' F_m0s-1kg0 [0.1]
    E_m0s-1kg0 -> F_m0s-1kg0 [0.8]
    F_m0s-1kg0 -> F_m0s-1kg0 '*' T_m0s0kg0 [0.3]
    F_m0s-1kg0 -> T_m0s-1kg0 [0.7]
    T_m0s-1kg0 -> '(' E_m0s-1kg0 ')' [0.2]
    T_m0s-1kg0 -> 'omega' [0.8]
    
    E_m0s1kg0 -> E_m0s1kg0 '+' F_m0s1kg0 [0.1]
    E_m0s1kg0 -> E_m0s1kg0 '-' F_m0s1kg0 [0.1]
    E_m0s1kg0 -> F_m0s1kg0 [0.8]
    F_m0s1kg0 -> F_m0s1kg0 '*' T_m0s0kg0 [0.3]
    F_m0s1kg0 -> T_m0s1kg0 [0.7]
    T_m0s1kg0 -> '(' E_m0s1kg0 ')' [0.2]
    T_m0s1kg0 -> 't' [0.8]
    
    E_m0s0kg0 -> E_m0s0kg0 '+' F_m0s0kg0 [0.3]
    E_m0s0kg0 -> E_m0s0kg0 '-' F_m0s0kg0 [0.1]
    E_m0s0kg0 -> F_m0s0kg0 [0.6]
    F_m0s0kg0 -> F_m0s-1kg0 '*' T_m0s1kg0 [0.3]
    F_m0s0kg0 -> F_m0s0kg0 '*' T_m0s0kg0 [0.1]
    F_m0s0kg0 -> T_m0s0kg0 [0.6]
    T_m0s0kg0 -> F [0.2]
    T_m0s0kg0 -> 'alpha0' [0.6]
    T_m0s0kg0 -> 'C' [0.2]
    F -> '(' E_m0s0kg0 ')' [0.5]
    F -> 'sin(' E_m0s0kg0 ')' [0.5]"""

simboli_nihanje = {"x": ['x0', 'omega', 't', 'alpha0'], "start": "S", "const": "C"}

In [19]:
resi_z_gramatiko("nihanje.csv", "x", gramatika_nihanje, simboli_nihanje, 1000)            

##################### Kandidati: #####################
x0
2*x0
alpha0*x0 - x0
alpha0*x0 - x0
alpha0*x0 - x0
alpha0*x0 - x0
alpha0*x0 - x0
alpha0**2*x0
C0*alpha0*x0
C0*alpha0*x0
C0*alpha0*x0
C0*alpha0*x0
C0*alpha0*x0
C0*alpha0*x0
C0*alpha0*x0 + alpha0*x0
C0*alpha0*x0 + alpha0*x0
C0*alpha0*x0 + alpha0*x0
C0*alpha0*x0 + alpha0*x0
alpha0*x0*sin(alpha0)
alpha0*x0*sin(alpha0)
C0*alpha0**2*x0
C0*alpha0**2*x0
C0*alpha0**2*x0
C0*x0*sin(alpha0)
alpha0*x0*sin(C0 + omega*t)
alpha0*x0*sin(C0 + omega*t)
alpha0*x0*sin(C0 + omega*t)
alpha0*x0*sin(C0 + omega*t)
alpha0*x0*sin(C0 + omega*t)
C0*alpha0*x0 + x0
C0*alpha0*x0 + x0
C0*alpha0*x0 + x0
C0*alpha0*x0 + x0
alpha0**2*x0 + alpha0*x0
C0*alpha0**2*(C1*alpha0*x0 + alpha0**2*x0) - x0
C0*alpha0**2*(C1*alpha0*x0 + alpha0**2*x0) - x0
C0*alpha0**2*(C1*alpha0*x0 + alpha0**2*x0) - x0
alpha0*x0*sin(omega*t) - alpha0*x0
alpha0*x0*sin(omega*t) - alpha0*x0
alpha0**3*x0*(C0 + omega*t) + alpha0*x0
alpha0**3*x0*(C0 + omega*t) + alpha0*x0
C0*alpha0**2*x0*sin(alpha0)
C0

C0*x0*sin(alpha0**2*omega*t*sin(C1 + alpha0*omega*(alpha0*t + t) + omega*t*(C2 - alpha0 - omega*t)))
C0*alpha0**3*x0
C0*alpha0**3*x0
C0*alpha0**3*x0
alpha0*x0*sin(sin(C0*alpha0 + alpha0 + 3*omega*t))
alpha0*x0*sin(sin(C0*alpha0 + alpha0 + 3*omega*t))
alpha0*x0*sin(sin(C0*alpha0 + alpha0 + 3*omega*t))
alpha0*x0*sin(sin(C0*alpha0 + alpha0 + 3*omega*t))
alpha0*x0*sin(sin(C0*alpha0 + alpha0 + 3*omega*t))
C0*alpha0*x0*(C1*sin(alpha0) + omega*t)
alpha0*x0 + 3*x0
alpha0**3*x0*sin(alpha0*omega*t) + alpha0*x0 + x0
alpha0**3*x0*sin(alpha0*omega*t) + alpha0*x0 + x0
alpha0**3*x0*sin(alpha0*omega*t) + alpha0*x0 + x0
alpha0**3*x0*sin(alpha0*omega*t) + alpha0*x0 + x0
x0*sin(omega*t)
x0*sin(omega*t)
x0*sin(omega*t)
x0*sin(omega*t)
C0*x0 + x0*sin(omega*t) + x0
C0*x0 + x0*sin(omega*t) + x0
C0*x0 + x0*sin(omega*t) + x0
-alpha0*omega*t*x0 + x0*sin(omega*t)
alpha0*(alpha0*x0*sin(2*omega*t) - alpha0*x0)
alpha0*(alpha0*x0*sin(2*omega*t) - alpha0*x0)
alpha0*(alpha0*x0*sin(2*omega*t) - alpha0*x0)
alpha0*(alpha

2*x0*sin(C0 + alpha0**3*omega*t*sin(C1*alpha0)*sin(C2*alpha0*t*(alpha0**2*omega + omega) + alpha0 + omega*t)) - x0
2*x0*sin(C0 + alpha0**3*omega*t*sin(C1*alpha0)*sin(C2*alpha0*t*(alpha0**2*omega + omega) + alpha0 + omega*t)) - x0
2*x0*sin(C0 + alpha0**3*omega*t*sin(C1*alpha0)*sin(C2*alpha0*t*(alpha0**2*omega + omega) + alpha0 + omega*t)) - x0
2*x0*sin(C0 + alpha0**3*omega*t*sin(C1*alpha0)*sin(C2*alpha0*t*(alpha0**2*omega + omega) + alpha0 + omega*t)) - x0
C0*alpha0*x0 + omega*t*x0 + x0
C0*alpha0*x0 + omega*t*x0 + x0
C0*alpha0*x0 + omega*t*x0 + x0
alpha0**2*omega*t*x0
alpha0**2*omega*t*x0
alpha0**2*x0 + 2*alpha0*x0*sin(alpha0) + x0
alpha0**2*x0 + 2*alpha0*x0*sin(alpha0) + x0
x0*(C0 + alpha0**3*omega*(alpha0*t + t*sin(C1 + 2*alpha0 + 2*omega*t)))
x0*(C0 + alpha0**3*omega*(alpha0*t + t*sin(C1 + 2*alpha0 + 2*omega*t)))
alpha0*x0 + x0*sin(omega*t) + 2*x0
alpha0*x0 + x0*sin(omega*t) + 2*x0
C0*alpha0*(alpha0**2*x0 - alpha0*x0*sin(C1 + alpha0))
x0*(C0*alpha0 + alpha0 - omega*t)
x0*(C0*alpha0 +

C0*x0 + 2*x0
C0*x0 + 2*x0
2*alpha0**3*x0*sin(omega*t)
2*alpha0**3*x0*sin(omega*t)
alpha0*x0*(alpha0*omega*t*(C0 + alpha0) + 2*alpha0)
alpha0*x0*(alpha0*omega*t*(C0 + alpha0) + 2*alpha0)
alpha0*x0*(alpha0*omega*t*(C0 + alpha0) + 2*alpha0)
alpha0*x0*(alpha0*omega*t*(C0 + alpha0) + 2*alpha0)
alpha0*x0*(alpha0*omega*t*(C0 + alpha0) + 2*alpha0)
alpha0*x0*(alpha0*omega*t*(C0 + alpha0) + 2*alpha0)
alpha0*x0*(alpha0*omega*t*(C0 + alpha0) + 2*alpha0)
alpha0*x0*(alpha0*omega*t*(C0 + alpha0) + 2*alpha0)
alpha0*x0*(alpha0*omega*t*(C0 + alpha0) + 2*alpha0)
alpha0**2*x0*sin(sin(2*omega*t))
alpha0**2*x0*sin(sin(2*omega*t))
alpha0**2*x0*sin(sin(2*omega*t))
alpha0**2*x0*sin(sin(2*omega*t))
alpha0**2*x0*sin(sin(2*omega*t))
alpha0**2*x0*sin(sin(2*omega*t))
alpha0**2*x0*sin(sin(2*omega*t))
C0*alpha0**3*x0 + alpha0*x0
C0*alpha0**3*x0 + alpha0*x0
C0*alpha0**3*x0 + alpha0*x0
C0*alpha0**3*x0 + alpha0*x0
C0*alpha0**3*x0 + alpha0*x0
C0*alpha0**3*x0*sin(omega*t)
C0*alpha0**3*x0*sin(omega*t)
C0*alpha0**3*x0*sin(o

alpha0*x0*(C0 + alpha0*omega*t + 2*alpha0)
alpha0*x0*(C0 + alpha0*omega*t + 2*alpha0)
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
C0*alpha0*x0*sin(sin(alpha0 - omega*t))
x0*(alpha0 + omega*t - sin(C0 + alpha0*omega*t + alpha0))
x0*(alpha0 + omega*t - sin(C0 + alpha0*omega*t + alpha0))
x0*(alpha0 + omega*t - sin(C0 + alpha0*omega*t + alpha0))
C0*alpha0**3*x0*sin(alpha0*(alpha0 + omega*t) + alpha0)
C0*alpha0**3*x0*sin(alpha0*(alpha0 + omega*t) + alpha0)
C0*alpha0**3*x0*sin(alpha0*(alpha0 + omega*t) + alpha0)
C0*alpha0**3*x0*sin(alpha0*(alpha0 + omega*t) + alpha0)
C0*alpha0**3*x0*sin(alpha0*(alpha0 +

# 3. RAČUNANJE VERJETNOSTI

## 3.2 Enakomerno pospešeno gibanje

In [20]:
def verjetnost(ime_ciljne, gramatika, simboli, n_modelov):
    stevec = 1
    grammar = GeneratorGrammar(gramatika)
    modeli = generate_models(grammar, simboli, strategy_settings={"N": n_modelov}, verbosity=-1)
    for m in modeli:
        if m.__repr__() == "C0*a*t**2 + t*v0" or m.__repr__() == "C0*a*t**2 + v0*t" or m.__repr__() == "C0*a*t**2 + C1*t*v0" or m.__repr__() == "C0*a*t**2 + C1*v0*t" or m.__repr__() == "C0*t**2*a + t*v0" or m.__repr__() == "C0*t**2*a + v0*t" or m.__repr__() == "C0*t**2*a + C1*t*v0" or m.__repr__() == "C0*t**2*a + C1v0*t":
            stevec += 1
    print(f"Število pravilnih enačb v {n_modelov} modelih: {stevec}.")
    verj = 1-(1-stevec/n_primerov)**n_primerov
    print(f"Skupna verjetnost: {verj}.")

In [21]:
verjetnost("s", gramatika_enakomerno, simboli_enakomerno, 100000)            

Število pravilnih enačb v 100000 modelih: 3.
Skupna verjetnost: 0.9524474920745943.


## 3.3 Kosinusni izrek

In [22]:
def werjetnost(ime_ciljne, gramatika, simboli, n_modelov):
    stevec = 1
    grammar = GeneratorGrammar(gramatika)
    modeli = generate_models(grammar, simboli, strategy_settings={"N": n_modelov}, verbosity=-1)
    for m in modeli:
        if m.__repr__() == "sqrt(C0*a**2 + C1*b**2 - C2*a*b*cos(g))" or m.__repr__() == "sqrt(C0*b**2 + C1*a**2 - C2*a*b*cos(g))" or m.__repr__() == "sqrt(C0*a**2 + C1*b**2 - C2*b*a*cos(g))" or m.__repr__() == "sqrt(C0b**2 + C1*a**2 - C2*b*a*cos(g))"or m.__repr__() == "sqrt(- C0*b*a*cos(g) + C1*b**2 + C2*a**2)" or m.__repr__() == "sqrt(- C0*b*a*cos(g) + C1*a**2 + C2*b**2)"         or m.__repr__() == "sqrt(- C0*a*b*cos(g) + C1*b**2 + C2*a**2)" or m.__repr__() == "sqrt(- C0*a*b*cos(g) + C1*a**2 + C2*b**2)"            or m.__repr__() == "sqrt(C0*b**2 - C1*a*b*cos(g) +  C2*a**2)" or m.__repr__() == "sqrt(C0*a**2 - C1*a*b*cos(g) + C2*b**2)"          or m.__repr__() == "sqrt(C0*b**2 - C1*a*b*cos(g) + C2*a**2)" or m.__repr__() == "sqrt(C0*a**2 - C1*a*b*cos(g) +  C2*b**2)":            
            stevec += 1
    print(f"Število pravilnih enačb v {n_modelov} modelih: {stevec}.")
    verj = 1-(1-stevec/n_primerov)**n_primerov
    print(f"Skupna verjetnost: {verj}.")

In [23]:
verjetnost("c", gramatika_kosinusni, simboli_kosinusni, 100000)            

Število pravilnih enačb v 100000 modelih: 1.
Skupna verjetnost: 0.6339676587267709.


## 3.4 Sinusno nihanje

In [24]:
def modeli(ime_ciljne, gramatika, simboli, n_modelov):
    stevec = 1
    grammar = GeneratorGrammar(gramatika)
    modeli = generate_models(grammar, simboli, strategy_settings={"N": n_modelov}, verbosity=-1)
    for m in modeli:
        if m.__repr__() == "x0*sin(omega*t + alpha0)" or m.__repr__() == "sin(omega*t + alpha0)*x0" or m.__repr__() == "x0*sin(t*omega + alpha0)" or m.__repr__() == "x0*sin(alpha0 + omega*t)" or m.__repr__() == "x0*sin(alpha0 + t*omega)" or m.__repr__() == "sin(alpha0 + omega*t)*x0" or m.__repr__() == "sin(t*omega + alpha0)*x0" or m.__repr__() == "sin(omega*t + alpha0)*x0" or m.__repr__() == "C0*x0*sin(t*omega + alpha0)" or m.__repr__() == "x0*sin(alpha0 + C0*omega*t)" or m.__repr__() == "x0*sin(C0*t*omega + alpha0)" or m.__repr__() == "x0*sin(alpha0 + omega*t)":            
            stevec += 1
    print(f"Število pravilnih enačb v {n_modelov} modelih: {stevec}.")
    verj = 1-(1-stevec/n_primerov)**n_primerov
    print(f"Skupna verjetnost: {verj}.")

In [25]:
verjetnost("x", gramatika_nihanje, simboli_nihanje, 100000)            

Število pravilnih enačb v 100000 modelih: 1.
Skupna verjetnost: 0.6339676587267709.
